# Практическая работа 2 (Тест)
## Тема: Приведение данных в порядок

### Настройка окружения

In [159]:
import pandas as pd
import numpy as np

### Вопрос 1


Набор данных [BikesData.csv](./BikesData.csv) содержит сведения о работе велопроката.

Описание переменных:

- **Date** - день/месяц/год (с 1 декабря 2017 года по 30 ноября 2018 года)
- **Hour** - час (от 0 до 24)
- **Temperature** - температура в градусах Цельсия
- **Humidity** - процент влажности
- **Wind speed** - скорость ветра м/сек
- **Rainfall** - осадки в виде дождя, мм
- **Snowfall** - осадки в виде снега, мм
- **Seasons** - сезон (Winter, Spring, Summer, Autumn)
- **Holiday** - рабочий или праздничный день (Holiday/No holiday)
- **Function Day** - рабочий или нерабочий день велопроката (Yes/No)
- **Rental Count** - количество велосипедов, взятых в аренду в этот час

In [175]:
bikes = pd.read_csv("BikesData.csv", parse_dates=["Date"])
bikes.head()

,Date,Hour,Temperature,Humidity,Wind speed,Rainfall,Snowfall,Seasons,Holiday,Functioning Day,Rental Count
0,2017-12-01,0,-5.2,37,2.2,0.0,0.0,Winter,0,True,257
1,2017-12-01,1,-5.5,38,0.8,0.0,0.0,Winter,0,True,219
2,2017-12-01,2,-6.0,39,1.0,0.0,0.0,Winter,0,True,162
3,2017-12-01,3,-6.2,40,0.9,0.0,0.0,Winter,0,True,148
4,2017-12-01,4,-6.0,36,2.3,0.0,0.0,Winter,0,True,97


1. Укажите кол-во наблюдений в датафрейме.

In [176]:
len(bikes)

8760

2. Укажите кол-во переменных, содержащих пропуски.

In [177]:
len(bikes.columns[bikes.isnull().sum() > 0])

1

3. Укажите кол-во пропусков в переменной **Temperature**.

In [178]:
bikes.isnull().sum().loc["Temperature"]

np.int64(179)

4. Укажите среднее значение температуры до заполнения пропусков. Ответ округлите до 2 знаков после запятой.

In [179]:
round(bikes.Temperature.mean(), 2)

np.float64(12.88)

5. Укажите медианное значение температуры до заполнения пропусков.

In [180]:
bikes.Temperature.median()

np.float64(13.7)

6. Создайте столбец **Temp_26** и заполните все пропуски данных температуры значением 26. Укажите среднее значение температуры (столбец **Temp_26**) в 2018 году. Ответ округлите до 2 знаков после запятой.

In [181]:
bikes["Temp_26"] = bikes.Temperature.fillna(26)
mask = bikes.Date.dt.year == 2018

round(bikes[mask]["Temp_26"].mean(), 2)

np.float64(14.49)

7. Создайте столбец **Temp_Median** и заполните все пропуски данных температуры медианным значением. Укажите кол-во наблюдений, когда температура (столбец **Temp_Median**) превысила 24 градуса.

In [231]:
bikes["Temp_Median"] = bikes.Temperature.fillna(bikes.Temperature.median())
mask = bikes["Temp_Median"] > 24

mask.sum()

np.int64(1775)

8. Создайте столбец **Temperature_Random** и заполните все пропуски случайными значениями температуры из столбца **Temperature**. Укажите значение температуры (столбец **Temperature_Random**) для строки на 4302 позиции.

    *Примечание*: С помощью метода `np.random.choice()` предварительно создайте серию, в которой для каждого наблюдения будет храниться случайное значение температуры из столбца **Temperature** (значения NaN в списке, из которого будут выбираться случайные значения, необходимо исключить). Задайте начальное значение генератора случайных чисел равное 6. Затем используйте получившуюся серию для заполнения пропусков с помощью метода `fillna()`: если для какого-то наблюдения будет пропущено значение температуры, то пропуск заполняется значением из серии, которое соответствует индексу наблюдения.

In [183]:
np.random.seed(6)
n = len(bikes)
temperatures = bikes.Temperature.dropna()

fill_values = pd.Series(np.random.choice(temperatures, n))
bikes["Temperature_Random"] = bikes.Temperature.fillna(fill_values)

bikes.iloc[4302]["Temperature_Random"]

np.float64(-4.7)

9. Напишите функцию, которая будет заполнять пропуски на основе среднего значения температуры на одну строку выше и на одну строку ниже пропущенного. Для заполнения пропусков создайте отдельный столбец. Укажите кол-во наблюдений, для которых не удалось заполнить пропуски по предложенному алгоритму.

In [ ]:
def fill_missing(
    df: pd.DataFrame, col: str, *, inplace: bool = False
) -> tuple[pd.DataFrame, int]:
    if not inplace:
        df = df.copy()

    filled_col = col + "_Filled"
    df[filled_col] = df[col]

    df[filled_col] = df[filled_col].fillna((df[col].shift(1) + df[col].shift(-1)) / 2)

    missing_after_fill = df[filled_col].isnull().sum()

    return df, missing_after_fill


_, missing_after_fill = fill_missing(bikes, "Temperature", inplace=True)
missing_after_fill

np.int64(6)

10. Создайте столбец **Date_Hour**, который будет хранить дату и время в формате: 2024-03-25 22:00:00. С помощью линейной интерполяции заполните пропущенные значения. Укажите температуру для строки, находящейся на 3539 позиции.

In [ ]:
bikes["Date_Hour"] = bikes.Date + pd.to_timedelta(bikes["Hour"], unit="h")

bikes.Temperature.interpolate()[3539]

np.float64(17.4)

### Вопрос 2
Продолжаем работать с набором данных [BikesData.csv](./BikesData.csv), который содержит сведения о работе велопроката.

Заполните пропуски в столбце **Temperature** с использованием линейной интерполяции (см. п.10 вопроса 1) и выполните следующие задания:

In [186]:
bikes.Temperature = bikes.Temperature.interpolate()

1. Создайте бинарную переменную (столбец **Normal Humidity**) для хранения информации о том, была ли влажность нормальной (от 40% до 60%). Укажите кол-во наблюдений в 2017 году с ненормальной влажностью.

In [237]:
bikes["Normal Humidity"] = np.where(bikes.Humidity.between(40, 60), 1, 0)
mask = (bikes.Date.dt.year == 2017) & (bikes["Normal Humidity"] == 0)

mask.sum()

np.int64(514)

2. Создайте категориальную переменную **Temperature Category** перекодировав количественную **Temperature** по следующей шкале:

    - *t < 0* - категория Freezing (заморозки),
    - *0 <= t < 15* - категория Chilly (прохладно),
    - *15 <= t < 25* - категория Nice (идеальная),
    - *t >= 25* - категория Hot (жарко).

    Укажите кол-во наблюдений в категории Chilly.

In [ ]:
bikes["Temperature Category"] = pd.cut(
    bikes.Temperature,
    bins=[-np.inf, 0, 15, 25, np.inf],
    labels=["Freezing", "Chilly", "Nice", "Hot"],
    right=False,
)

mask = bikes["Temperature Category"] == "Chilly"

mask.sum()

np.int64(3190)

3. Так как измерения ведутся каждый час, определим, к какой температурной категории относится весь день. Будем считать, что день был, например, жарким, если большая часть наблюдений в течении дня относилась к категории Hot.

    Укажите кол-во прохладных дней в 2018 году.

In [ ]:
bikes_2018 = bikes[bikes.Date.dt.year == 2018]
daily_categories = bikes_2018.groupby("Date")["Temperature Category"].agg(
    lambda x: x.mode()[0]
)

mask = daily_categories == "Chilly"
mask.sum()

np.int64(128)

4. Создайте бинарную переменную **Good Weather**, показывающая является ли погода для данного наблюдения хорошей. Хорошая погода зависит от:

    - температуры (*Temperature Category = Nice*)
    - влажности (*Normal Humidity = 1*),
    - скорости ветра (*Wind speed <= 5.4*),
    - был ли дождь (*Rainfall = 0*),
    - был ли снег (*Snowfall = 0*).

    *Примечание*: для записи условия можно воспользоваться функцией `numpy.where`(условие, 1, 0).

    Укажите кол-во наблюдений с хорошей погодой весной 2018 года.

In [ ]:
mask = (
    (bikes["Temperature Category"] == "Nice")
    & (bikes["Normal Humidity"] == True)
    & (bikes["Wind speed"] <= 5.4)
    & (bikes.Rainfall == 0)
    & (bikes.Snowfall == 0)
)

bikes["Good Weather"] = np.where(mask, 1, 0)

spring_2018 = bikes[
    (bikes.Date.dt.year == 2018) & (bikes.Date.dt.month.isin([3, 4, 5]))
]

mask = spring_2018["Good Weather"] == 1

mask.sum()

np.int64(258)

5. Так как измерения ведутся каждый час, определим, была ли погода хорошей для всего дня, если для 12 и более наблюдений в течении дня погода была хорошей.

    Укажите, кол-во дней с хорошей погодой весной 2018 года.

In [258]:
daily_categories = spring_2018.groupby("Date")["Good Weather"].sum()

mask = daily_categories >= 12

mask.sum()

np.int64(3)

### Вопрос 3

Набор данных [students.csv](./students.csv) содержит сведения о студентах, которые проходили анкетирование.

In [ ]:
students = pd.read_csv("students.csv")
students.head()

,Age,Growth,Shoe size,Course number,Year of birth,Friend number,Russian rating,Maths rating,Physics rating,Computer science rating,...,Social network duration min,Chocolate,City population,Strange people,Your insitute,Brother-sister,Plane seat,MIddle and index finger,Middle and ring finger,Middle and little finger
0,20,170,40,2,2002,100,85,86,0,84,...,45,Сникерс,800000.0,Экономический,Математический и ИТ,нет ни брата ни сестры,У окна (пардон: у иллюминатора),20,10.0,40.0
1,22,191,43,7,2000,307,69,74,64,72,...,180,КитКат,800000.0,Математический и ИТ,Математический и ИТ,есть и брат и сестра,В середине,5,5.0,20.0
2,21,172,41,4,2000,186,78,62,0,0,...,90,КитКат,770000.0,Педагогический,Другое,есть брат,У окна (пардон: у иллюминатора),13,10.0,26.0
3,19,168,38,2,2003,604,72,0,0,0,...,200,Баунти,900000.0,СоцГум,Юридический,есть сестра,У окна (пардон: у иллюминатора),12,13.5,35.0
4,23,159,36,1,1998,144,0,0,0,0,...,400,Баунти,860000.0,Педагогический,Педагогический,есть сестра,У окна (пардон: у иллюминатора),10,11.0,22.0


1. Укажите кол-во строк датафрейма.

In [294]:
students.shape[0]

186

2. Укажите кол-во переменных, содержащих пропуски.

In [295]:
students.columns[students.isnull().any()].shape[0]

4

3. Удалите строки (не изменяя исходный датафрейм), в которых содержится хотя бы один пропуск. Сколько всего осталось строк?

In [296]:
students.dropna().shape[0]

118

4. Удалите столбцы (не изменяя исходный датафрейм), в которых есть 10 и более пропусков. Сколько всего осталось столбцов?

In [301]:
students.dropna(thresh=students.shape[0] - 10, axis=1).shape[1]

45

5. Заполним пропуски в признаке **Weight** (вес): всем женщинам (поле **Sex** хранит пол студента), которые не указали свой вес, присвоим средний вес женщин, а мужчинам - средний вес мужчин. Укажите средний вес всех студентов после заполнения пропусков с точностью до 2 знаков после запятой.

In [314]:
is_woman = students.Sex == "женский"

women_mean_weight = students[is_woman].Weight.mean()
men_mean_weight = students[~is_woman].Weight.mean()

means = pd.Series(np.where(is_woman, women_mean_weight, men_mean_weight))

students.Weight.fillna(means, inplace=True)

round(students.Weight.mean(), 2)

np.float64(63.53)

### Вопрос 4
С использованием следующего программного кода создайте датафрейм:

Укажите кол-во дублирующихся строк.

In [ ]:
# Код из вопроса
np.random.seed(1)
x_cnt = np.random.randint(100, 200)
y_cnt = np.random.randint(150, 300)
data = pd.DataFrame(
    {"a": ["x"] * x_cnt + ["y"] * y_cnt, "b": np.random.randint(1, 100, x_cnt + y_cnt)}
)

data.duplicated().sum()

np.int64(258)